In [1]:
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
_pipeline_root = 'pipeline'
context = InteractiveContext(pipeline_root=_pipeline_root)

2023-04-06 14:33:11.953075: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 14:33:12.074716: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-06 14:33:12.074736: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-06 14:33:12.649977: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
from tfx.proto import example_gen_pb2
import os
from tfx.components import CsvExampleGen


output= example_gen_pb2.Output(
split_config=example_gen_pb2.SplitConfig(splits=[
    example_gen_pb2.SplitConfig.Split(name='train',hash_buckets=7),
    example_gen_pb2.SplitConfig.Split(name='eval',hash_buckets=3)
    ])   
)

examples_gen = CsvExampleGen(input_base='data',output_config=output)
context.run(examples_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [3]:
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(
    examples=examples_gen.outputs['examples']
)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [4]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True
)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [5]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'ALT',INT,required,,-
'AST',INT,required,,-
'Cholesterol',INT,required,,-
'Gtp',INT,required,,-
'HDL',INT,required,,-
'LDL',INT,required,,-
'Urine protein',INT,required,,-
'age',INT,required,,-
'dental caries',INT,required,,-


In [6]:
from tfx.components import ExampleValidator

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [7]:
smoking_constants_file = 'smoking_constants.py'

In [9]:
%%writefile {smoking_constants_file}

SCALE_TO_Z_SCORE = {'ALT':None,'AST':None,'Cholesterol':None,'Gtp':None,'HDL':None,'LDL':None,'fasting blood sugar':None,'hemoglobin':None,'relaxation':None,'serum creatinine':None,'systolic':None,'triglyceride':None}
NORMALIZE_TO_0_1 = {'age':None,'height(cm)':None,'waist(cm)':None,'weight(kg)':None}
CLIP_AFTER = {'hearing(left)':1,'hearing(right)':1,'eyesight(left)':2,'eyesight(right)':2,'Urine protein':1}
UNCHANGED = {'dental caries':None,'smoking':None}

def transformed_name(key):
    return key + '_xf'


Overwriting smoking_constants.py


In [8]:
smoking_transform_file = 'smoking_transform.py'

In [11]:
%%writefile {smoking_transform_file}

import tensorflow as tf
import tensorflow_transform as tft

import smoking_constants

SCALE_TO_Z_SCORE = smoking_constants.SCALE_TO_Z_SCORE
transformed_name = smoking_constants.transformed_name
NORMALIZE_TO_0_1 = smoking_constants.NORMALIZE_TO_0_1
CLIP_AFTER = smoking_constants.CLIP_AFTER
UNCHANGED = smoking_constants.UNCHANGED


def preprocessing_fn(inputs):
    outputs = {}
    for key in SCALE_TO_Z_SCORE.keys():
        key_data = inputs[key]
        scaled_data = tft.scale_to_z_score(key_data)
        clipped_data = tf.clip_by_value(scaled_data, -3.0,3.0)
        outputs[transformed_name(key)] = clipped_data

    for key in NORMALIZE_TO_0_1.keys():
        key_data2 = inputs[key]
        scaled_data2 = tft.scale_to_0_1(key_data2)
        outputs[transformed_name(key)] = scaled_data2

    for key in CLIP_AFTER.keys():
        key_data3 = inputs[key]
        clipped_data3 = tf.clip_by_value(key_data3, 0,CLIP_AFTER[key])
        scaled_data3 = tft.scale_to_0_1(clipped_data3)
        outputs[transformed_name(key)] = scaled_data3

    for key in UNCHANGED.keys():
        key_data4 = inputs[key]
        outputs[transformed_name(key)] = key_data4

    return outputs

Overwriting smoking_transform.py


In [9]:
import tfx
transform = tfx.components.Transform(
    examples = examples_gen.outputs['examples'],
    schema = schema_gen.outputs['schema'],
    module_file=os.path.abspath(smoking_transform_file)
)
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying smoking_constants.py -> build/lib
copying trainer.py -> build/lib
copying smoking_transform.py -> build/lib
copying smoking_trainer.py -> build/lib
installing to /tmp/tmpqnk6qvr_
running install
running install_lib
copying build/lib/smoking_trainer.py -> /tmp/tmpqnk6qvr_
copying build/lib/smoking_transform.py -> /tmp/tmpqnk6qvr_
copying build/lib/trainer.py -> /tmp/tmpqnk6qvr_
copying build/lib/smoking_constants.py -> /tmp/tmpqnk6qvr_
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info


/home/thakur/anaconda3/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpqnk6qvr_/tfx_user_code_Transform-0.0+4680740557ed5c99a703ab8512e07214f1a72a08400f1ed1bd0137a790716046-py3.9.egg-info
running install_scripts
creating /tmp/tmpqnk6qvr_/tfx_user_code_Transform-0.0+4680740557ed5c99a703ab8512e07214f1a72a08400f1ed1bd0137a790716046.dist-info/WHEEL
creating '/tmp/tmpq_77pd8c/tfx_user_code_Transform-0.0+4680740557ed5c99a703ab8512e07214f1a72a08400f1ed1bd0137a790716046-py3-none-any.whl' and adding '/tmp/tmpqnk6qvr_' to it
adding 'smoking_constants.py'
adding 'smoking_trainer.py'
adding 'smoking_transform.p

2023-04-06 14:34:07.393249: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-06 14:34:07.393269: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-06 14:34:07.393296: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mako02.fysik.su.se): /proc/driver/nvidia/version does not exist
2023-04-06 14:34:07.393467: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/65ed32e20f394abc956e919bddc6caa2/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [10]:
trainer_module_file = 'smoking_trainer.py'

In [11]:
%%writefile {trainer_module_file}

import tensorflow as tf
# import tensorflow_decision_forests as tfdf
import tensorflow_transform as tft

from absl import logging
from tensorflow.keras import layers, Model, optimizers, losses, metrics
from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from typing import List, Text
import smoking_constants
import os

SCALE_TO_Z_SCORE = smoking_constants.SCALE_TO_Z_SCORE
transformed_name = smoking_constants.transformed_name
NORMALIZE_TO_0_1 = smoking_constants.NORMALIZE_TO_0_1
CLIP_AFTER = smoking_constants.CLIP_AFTER
UNCHANGED = smoking_constants.UNCHANGED

def model_fn():
    inputs = {}
    for key,item in SCALE_TO_Z_SCORE.items():
        inputs[transformed_name(key)] = tf.keras.Input(shape=(1,),name=transformed_name(key))

    for key,item in NORMALIZE_TO_0_1.items():
        inputs[transformed_name(key)] = tf.keras.Input(shape=(1,),name=transformed_name(key))

    for key,item in CLIP_AFTER.items():
        inputs[transformed_name(key)] = tf.keras.Input(shape=(1,),name=transformed_name(key))

    inputs['dental caries_xf'] = tf.keras.Input(shape=(1,),name=transformed_name('dental caries'))
    output = tf.keras.layers.Concatenate()(tf.nest.flatten(inputs))
    output = tf.keras.layers.BatchNormalization()(output)
    output = tf.keras.layers.Dense(32, activation='relu')(output)
    output = tf.keras.layers.Dropout(0.3)(output)
    output = tf.keras.layers.BatchNormalization()(output)
    output = tf.keras.layers.Dense(32, activation='relu')(output)
    output = tf.keras.layers.Dropout(0.3)(output)
    output = tf.keras.layers.BatchNormalization()(output)
    output = tf.keras.layers.Dense(32, activation='relu')(output)
    output = tf.keras.layers.Dropout(0.3)(output)
    output = tf.keras.layers.BatchNormalization()(output)
    output = tf.keras.layers.Dense(32, activation='relu')(output)
    output = tf.keras.layers.Dropout(0.3)(output)
    output = tf.keras.layers.BatchNormalization()(output)
    output = tf.keras.layers.Dense(32, activation='relu')(output)
    output = tf.keras.layers.Dropout(0.3)(output)
    output = tf.keras.layers.BatchNormalization()(output)
    output = tf.keras.layers.Dense(32, activation='relu')(output)
    output = tf.keras.layers.Dropout(0.3)(output)
    output = tf.keras.layers.BatchNormalization()(output)
    output = tf.keras.layers.Dense(16, activation='relu')(output)
    output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
    model = tf.keras.models.Model(inputs,output)

    # Compile model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                loss='binary_crossentropy',#mae
                metrics=['binary_accuracy'])

    return model





LABEL_KEY = 'smoking_xf'

def _gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames,compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, batch_size=512):
    transformed_feature_spec = tf_transform_output.transformed_feature_spec().copy()
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        label_key=LABEL_KEY
    )
    return dataset

def _get_serve_tf_examples_fn(model, tf_transform_output):
    """
    Returns a function that parses a serialized tf.Example and applies
    the transformations during inference.
    Args:
        model: The model that we are serving.
        tf_transform_output: The transformation output that we want to 
            include with the model.
    """
    
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")])
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        
        required_feature_spec = {
            k: v for k, v in feature_spec.items() if k != 'smoking'
        }
        print(required_feature_spec)
        parsed_features = tf.io.parse_example(
            serialized_tf_examples,
            required_feature_spec
        )

        transformed_features = model.tft_layer(parsed_features)

        return model(transformed_features)

    return serve_tf_examples_fn



def run_fn(fn_args):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    train_dataset = input_fn(fn_args.train_files,tf_transform_output)
    eval_dataset = input_fn(fn_args.eval_files,tf_transform_output)

    model = model_fn()

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir),'logs')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,update_freq='batch')

    model.fit(
        x=train_dataset,
        steps_per_epoch = fn_args.train_steps,
        validation_data = eval_dataset,
        validation_steps = fn_args.eval_steps,
        epochs=50,
        callbacks=[tensorboard_callback]
    )
    
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, 
                                 tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples')),
    }
    model.save(fn_args.serving_model_dir,save_format='tf',signatures=signatures,overwrite=True)#
    # print("Model saved to:", fn_args.serving_model_dir)


Overwriting smoking_trainer.py


In [12]:
from tfx.components import Trainer
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.proto import trainer_pb2
import tensorflow_transform as tft

TRAINING_STEPS = 1000
EVALUTION_STEPS = 100

trainer = Trainer(
    module_file=trainer_module_file,
    custom_executor_spec = executor_spec.ExecutorClassSpec(GenericExecutor),
    transform_graph = transform.outputs['transform_graph'],
    transformed_examples = transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    train_args = trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args = trainer_pb2.EvalArgs(num_steps=EVALUTION_STEPS)
)

context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying smoking_constants.py -> build/lib
copying trainer.py -> build/lib
copying smoking_transform.py -> build/lib
copying smoking_trainer.py -> build/lib
installing to /tmp/tmpynseszqh
running install
running install_lib
copying build/lib/smoking_trainer.py -> /tmp/tmpynseszqh
copying build/lib/smoking_transform.py -> /tmp/tmpynseszqh
copying build/lib/trainer.py -> /tmp/tmpynseszqh
copying build/lib/smoking_constants.py -> /tmp/tmpynseszqh
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'


/home/thakur/anaconda3/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpynseszqh/tfx_user_code_Trainer-0.0+5c774f7185d983de4bb4c04b1171052df340977313a3f789121ec1581ab74fba-py3.9.egg-info
running install_scripts
creating /tmp/tmpynseszqh/tfx_user_code_Trainer-0.0+5c774f7185d983de4bb4c04b1171052df340977313a3f789121ec1581ab74fba.dist-info/WHEEL
creating '/tmp/tmp10l8yuxd/tfx_user_code_Trainer-0.0+5c774f7185d983de4bb4c04b1171052df340977313a3f789121ec1581ab74fba-py3-none-any.whl' and adding '/tmp/tmpynseszqh' to it
adding 'smoking_constants.py'
adding 'smoking_trainer.py'
adding 'smoking_transform.py'
adding 'trainer.py'
adding 'tfx_user_code_Trainer-0.0+5c774f7185d983de4bb4c04b1171052df340977313a3f789121ec1581ab74fba.dist-info/METADATA'
adding 'tfx_user_code_Trainer-0.0+5c774f7185d983de4bb4c04b1171052df340977313a3f789121ec1581ab74fba.dist-info/WHEEL'
adding 'tfx_user_code_Trainer

Processing ./pipeline/_wheels/tfx_user_code_Trainer-0.0+5c774f7185d983de4bb4c04b1171052df340977313a3f789121ec1581ab74fba-py3-none-any.whl
Epoch 1/50
1000/1000 [==============================] - 7s 4ms/step - loss: 0.4975 - binary_accuracy: 0.7350 - val_loss: 0.4765 - val_binary_accuracy: 0.7540
Epoch 2/50
1000/1000 [==============================] - 4s 4ms/step - loss: 0.4806 - binary_accuracy: 0.7500 - val_loss: 0.4854 - val_binary_accuracy: 0.7438
Epoch 3/50
1000/1000 [==============================] - 4s 4ms/step - loss: 0.4763 - binary_accuracy: 0.7535 - val_loss: 0.4759 - val_binary_accuracy: 0.7560
Epoch 4/50
1000/1000 [==============================] - 4s 4ms/step - loss: 0.4750 - binary_accuracy: 0.7542 - val_loss: 0.4778 - val_binary_accuracy: 0.7480
Epoch 5/50
1000/1000 [==============================] - 4s 4ms/step - loss: 0.4735 - binary_accuracy: 0.7558 - val_loss: 0.4765 - val_binary_accuracy: 0.7528
Epoch 6/50
1000/1000 [==============================] - 4s 4ms/step - lo

INFO:tensorflow:struct2tensor is not available.


...

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


{'ALT': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None), 'AST': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None), 'Cholesterol': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None), 'Gtp': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None), 'HDL': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None), 'LDL': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None), 'Urine protein': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None), 'age': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None), 'dental caries': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None), 'eyesight(left)': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'eyesight(right)': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'fasting blood sugar': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None), 'hearing(left)': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=No

INFO:tensorflow:Assets written to: pipeline/Trainer/model/6/Format-Serving/assets


INFO:tensorflow:Assets written to: pipeline/Trainer/model/6/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [13]:
model_path = trainer.outputs['model'].get()[0].uri

In [27]:
%reload_ext tensorboard
%tensorboard --logdir {model_path}

Reusing TensorBoard on port 6006 (pid 45778), started 0:12:43 ago. (Use '!kill 45778' to kill it.)

In [21]:
import tensorflow_model_analysis as tfma
eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=model_path,
    tags=[tf.saved_model.SERVING]
)

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='smoking')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
    tfma.MetricsSpec(metrics=[
    tfma.MetricConfig(class_name='BinaryAccuracy'),
    tfma.MetricConfig(class_name='ExampleCount'),
    tfma.MetricConfig(class_name='FalsePositives'),
    tfma.MetricConfig(class_name='TruePositives'),
    tfma.MetricConfig(class_name='FalseNegatives'),
    tfma.MetricConfig(class_name='TrueNegatives'),
    tfma.MetricConfig(class_name='Precision'),
    tfma.MetricConfig(class_name='Recall')
    ])
    ]
)

from tfx.components import Evaluator

evaluator = Evaluator(
     examples=examples_gen.outputs['examples'],
     model=trainer.outputs['model'],
     eval_config=eval_config)

context.run(evaluator, enable_cache=True) 

ExecutionResult(
    component_id: Evaluator
    execution_id: 10
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [22]:
eval_results = evaluator.outputs['evaluation'].get()[0].uri
tfma_results = tfma.load_eval_result(eval_results)

In [25]:
tfma.view.render_slicing_metrics(tfma_results)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [19]:
eval_results = evaluator.outputs['evaluation'].get()[0].uri
tfma_results = tfma.load_eval_result(eval_results)

tfma.view.render_slicing_metrics(tfma_results)

from tfx.components import Pusher
from tfx.proto import pusher_pb2

_serving_model_dir = 'Serve'

pusher = Pusher(
    model = trainer.outputs['model'],
    model_blessing = evaluator.outputs['blessing'],
    push_destination = pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
    base_directory=_serving_model_dir
    )
))
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 8
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

In [35]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

total 0


In [36]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

missing_thresholds: true



In [26]:
evaluator.outputs

{'evaluation': OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}),
 'blessing': OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={})}

In [39]:
pusher.outputs

{'pushed_model': OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={})}

In [45]:
# push_uri = pusher.outputs['pushed_model'].get()[0].uri
# model = tf.saved_model.load(push_uri)

# for item in model.signatures.items():
#   pp.pprint(item)

In [32]:
import requests
import json
import base64
# The URL of the prediction endpoint
url = 'http://localhost:8501/v1/models/6:predict'

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

example = tf.train.Example(features=tf.train.Features(feature={
'ALT': _int64_feature(0),
'AST': _int64_feature(0),
'Cholesterol': _int64_feature(0),
'Gtp': _int64_feature(0),
'HDL': _int64_feature(0),
'LDL': _int64_feature(0),
'fasting blood sugar': _int64_feature(0),
'hemoglobin': _float_feature(0),
'relaxation': _int64_feature(0),
'serum creatinine': _float_feature(0),
'systolic': _int64_feature(0),
'triglyceride': _int64_feature(0),
'age': _int64_feature(0),
'height(cm)': _int64_feature(0),
'waist(cm)': _float_feature(0),
'weight(kg)': _int64_feature(0),
'hearing(left)': _int64_feature(0),
'hearing(right)': _int64_feature(0),
'eyesight(left)': _float_feature(0),
'eyesight(right)': _float_feature(0),
'Urine protein': _int64_feature(0),
'dental caries': _int64_feature(0)
}))
serialized_example = example.SerializeToString()

example_bytes = base64.b64encode(serialized_example).decode('utf-8')
payload = {
    'signature_name': 'serving_default',
    'instances': [{
        'b64': example_bytes
    }]
}
response = requests.post(url, json=payload)

# Get the prediction results
prediction_results = json.loads(response.text)
print(round(prediction_results['predictions'][0][0]))

0
